In [2]:
import sys
import itertools
import logging
from math import sqrt
from operator import add
from os.path import join, isfile, dirname

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

import json

from pyspark.mllib.fpm import FPGrowth

In [3]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
print("Running Spark Version %s" % (spark.version))

#### load business data ###
# path_business ="yizhan/Desktop/cs181/dataset/buiness.json"
path_business = "dataset/business.json"
df_business_raw = spark.read.json(path_business)
catDF = df_business_raw.select(df_business_raw["categories"])
catDF_iter = catDF.rdd.collect()    

Running Spark Version 2.2.0


In [4]:
catDF.count()

156639

In [5]:
data = catDF.rdd.filter(lambda item : 'Restaurants' in item.categories ).map(lambda item: set([x for x in item.categories if x !='Restaurants' and x != 'Food'])).filter(lambda item: len(item) > 1)

In [6]:
data.count()

34557

In [7]:
count = 0
cat_iter = data.collect()
for each in cat_iter:  
    print(each)
    count += 1
    if count > 10:
        break 

{'Convenience Stores', 'Soul Food'}
{'Sports Bars', 'American (Traditional)', 'Nightlife', 'Burgers', 'Bars'}
{'Italian', 'Diners', 'Breakfast & Brunch', 'Sandwiches', 'Mexican'}
{'American (Traditional)', 'Seafood'}
{'Comfort Food', 'Canadian (New)', 'American (Traditional)', 'Nightlife', 'Bars'}
{'Taiwanese', 'Coffee & Tea'}
{'Chicken Wings', 'American (Traditional)', 'Nightlife', 'Burgers', 'Bars'}
{'Irish', 'Nightlife', 'Pubs', 'Bars'}
{'Sandwiches', 'Chicken Wings', 'Pizza'}
{'Nightlife', 'Bars'}
{'Asian Fusion', 'Sushi Bars'}


In [8]:
# data.saveAsTextFile('category_data.csv')

In [9]:
model = FPGrowth.train(data.map(lambda item : list(item)), minSupport=0.001, numPartitions=10)
result = model.freqItemsets()
result_list = result.collect()

# Prelim Results from FP Growth Model

In [10]:
count = 0 
for fi in result_list:
    print(fi)
    count += 1
    if count > 15:
        break

FreqItemset(items=['Party & Event Planning'], freq=120)
FreqItemset(items=['Party & Event Planning', 'Caterers'], freq=40)
FreqItemset(items=['Party & Event Planning', 'Caterers', 'Event Planning & Services'], freq=40)
FreqItemset(items=['Party & Event Planning', 'Venues & Event Spaces'], freq=36)
FreqItemset(items=['Party & Event Planning', 'Venues & Event Spaces', 'Event Planning & Services'], freq=36)
FreqItemset(items=['Party & Event Planning', 'Event Planning & Services'], freq=120)
FreqItemset(items=['Juice Bars & Smoothies'], freq=582)
FreqItemset(items=['Juice Bars & Smoothies', 'American (New)'], freq=46)
FreqItemset(items=['Juice Bars & Smoothies', 'Coffee & Tea'], freq=145)
FreqItemset(items=['Juice Bars & Smoothies', 'Coffee & Tea', 'Sandwiches'], freq=37)
FreqItemset(items=['Juice Bars & Smoothies', 'Coffee & Tea', 'Cafes'], freq=58)
FreqItemset(items=['Juice Bars & Smoothies', 'Ice Cream & Frozen Yogurt'], freq=48)
FreqItemset(items=['Juice Bars & Smoothies', 'Desserts'],

In [11]:
cat_set_dict = {}
for item_set in result_list:
    main_label = item_set.items[0]
    current_freq = item_set.freq
    root_freq = current_freq
    if main_label in cat_set_dict:
        value = cat_set_dict[main_label]
        root_freq = value[0]
        label_set = value[1]
    else:
        label_set = [main_label]
        cat_set_dict[main_label] = [current_freq, label_set]
    
    if len(item_set) == 1:
        continue
    else:
        secondary_label = item_set.items[len(item_set.items)-1] # get the last label 
        if secondary_label not in label_set and current_freq > 0.5*root_freq:
            label_set.append(secondary_label)
            cat_set_dict[main_label] = [root_freq, label_set]
         
    

# Final Results with Frequent Appearing Category Combination

In [12]:
for k,v in cat_set_dict.items():
    print(k,v[1])

Party & Event Planning ['Party & Event Planning', 'Event Planning & Services']
Juice Bars & Smoothies ['Juice Bars & Smoothies']
Poke ['Poke', 'Hawaiian']
Fish & Chips ['Fish & Chips']
Sports Bars ['Sports Bars', 'American (Traditional)', 'Bars', 'Nightlife']
Kebab ['Kebab']
Dance Clubs ['Dance Clubs', 'Bars', 'Nightlife']
Greek ['Greek', 'Mediterranean']
Waffles ['Waffles', 'Breakfast & Brunch']
Hotels & Travel ['Hotels & Travel', 'Event Planning & Services']
Cafes ['Cafes']
Bowling ['Bowling', 'Active Life']
Health Markets ['Health Markets', 'Specialty Food']
Latin American ['Latin American']
Pool Halls ['Pool Halls', 'Bars', 'Nightlife']
Beer Bar ['Beer Bar', 'Bars', 'Nightlife']
Barbeque ['Barbeque']
Mongolian ['Mongolian']
Nightlife ['Nightlife']
Karaoke ['Karaoke', 'Bars', 'Nightlife']
Cantonese ['Cantonese', 'Chinese']
Arts & Entertainment ['Arts & Entertainment', 'Bars', 'Nightlife']
Music Venues ['Music Venues', 'Arts & Entertainment', 'Bars', 'Nightlife']
Coffee & Tea ['Coffe

In [14]:
len(cat_set_dict)

183